In [1]:
import json
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
d = None  
data = None  
filename = 'data_analysis.json'
with open(filename) as f:  
    data = f.read()  
    d = json.loads(data)

In [3]:
df = pd.DataFrame.from_dict(d)

In [4]:
del d

In [5]:
df.shape

(1000000, 10)

In [6]:
df.head()

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
0,10085 SCRIPPS RANCH CT STE A,44420000,SAN DIEGO,50 to 99,AMD CUSTOM,3123628000,$20 to 50 Million,CA,10+ years,92131
1,2566 SHALLOWFORD RD NE STE 104 # 302,31490000,ATLANTA,1 to 4,Real Hope Real Estate Inc,None,"Less Than $500,000",GA,10+ years,30345
2,212 E MAIN ST,53120000,NEOSHO,1 to 4,Jimmy Sexton Photography,4046331779,"Less Than $500,000",MO,10+ years,64850
3,6032 CHEROKEE DR,54000000,CINCINNATI,1 to 4,YOU'RE ART,4174513798,"Less Than $500,000",OH,10+ years,45243
4,1315 N WOOSTER AVE,54100000,STRASBURG,1 to 4,Hayberg Restoration Network LLC,5135612584,"$500,000 to $1 Million",OH,10+ years,44680


### Part1 : Fill rate for each field

In [8]:
df.notnull().sum()

address             999986
category_code       999986
city                999986
headcount           962352
name                999986
phone               590889
revenue             943092
state               999986
time_in_business    916125
zip                 999988
dtype: int64

### Part2 : True-valued Fill rate

In [9]:
#chekc distinct value type in each column
for col in df.columns:
    print (col, set([type(val) for val in list(df[col].unique())]) )

address {<class 'int'>, <class 'NoneType'>, <class 'str'>}
category_code {<class 'int'>, <class 'NoneType'>, <class 'str'>}
city {<class 'NoneType'>, <class 'int'>, <class 'str'>}
headcount {<class 'int'>, <class 'NoneType'>, <class 'str'>}
name {<class 'NoneType'>, <class 'int'>, <class 'str'>}
phone {<class 'int'>, <class 'NoneType'>, <class 'str'>}
revenue {<class 'int'>, <class 'NoneType'>, <class 'str'>}
state {<class 'int'>, <class 'NoneType'>, <class 'str'>}
time_in_business {<class 'int'>, <class 'NoneType'>, <class 'str'>}
zip {<class 'int'>, <class 'NoneType'>, <class 'str'>}


###### Count of true-valued for 'state'

First lets check the unique values in the column 'state'.

In [10]:
df['state'].unique()

array(['CA', 'GA', 'MO', 'OH', 'AL', 'MI', 'ME', 'FL', 'NM', 'HI', 'IN',
       'NY', 'IA', 'KS', 'MD', 'NJ', 'VA', 'DC', 'OR', 'NC', 'TX', 'PA',
       'ID', 'WA', 'UT', 'TN', 'MA', 'CT', 'CO', 'AZ', 'MS', 'OK', 'KY',
       'LA', 'MN', 'ND', 'WV', 'WI', 'SC', 'NH', 'IL', 'MT', 'RI', 'AR',
       'NV', 'DE', 'WY', 'SD', 'NE', 'VT', 'AK', 'none', ' ', '0', None,
       'VI', '', 'null', 'PR', 0], dtype=object)

In [85]:
def valid_state_cnt(x):
    '''for each row in the data frame it checks if the state has valid/meaningfull value 
    and returns just the rows with '''
    valid_states = ['CA', 'GA', 'MO', 'OH', 'AL', 'MI', 'ME', 'FL', 'NM', 'HI', 'IN',
       'NY', 'IA', 'KS', 'MD', 'NJ', 'VA', 'DC', 'OR', 'NC', 'TX', 'PA',
       'ID', 'WA', 'UT', 'TN', 'MA', 'CT', 'CO', 'AZ', 'MS', 'OK', 'KY',
       'LA', 'MN', 'ND', 'WV', 'WI', 'SC', 'NH', 'IL', 'MT', 'RI', 'AR',
       'NV', 'DE', 'WY', 'SD', 'NE', 'VT', 'AK',
       'VI', 'PR']
    return x[x['state'].isin(valid_states)]


In [88]:
print ('the number of record with valid states is ' + str(len(valid_state_cnt(df))))

the number of record with valid states is 999896


###### Count of true-valud for 'zip'

First let check the unique values in the column 'zip'.

In [16]:
set([type(z) for z in list(df['zip'].unique())])

{int, NoneType, str}

In [17]:
#subset of the data frame where zip is str
df_zip_str = df[df['zip'].apply(lambda x: type(x) in [type('3')])]
#subset of the data frame where zip is int
df_zip_int = df[df['zip'].apply(lambda x: type(x) in [type(3)])]
#subset of the data frame where zip is None
df_zip_none = df[df['zip'].apply(lambda x: type(x) in [type(None)])]

In [18]:
len(df) == len(df_zip_str) + len(df_zip_int) + len(df_zip_none)

True

In [21]:
# when zip is int it is clearly invalid
df_zip_int

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
116097,6200 S TRANSIT RD,22000000,LOCKPORT,1 to 4,In Harmony Pediatric Therapy,None,$20 to 50 Million,NY,10+ years,0
125452,7657 GALLEON WAY,71140000,CARLSBAD,10 to 19,Harrison's Marine,None,$1 to 2.5 Million,CA,10+ years,0
211479,625 S MAIN ST,23820000,KANNAPOLIS,1 to 4,OVERTON SECURITY SERVICES INC,4807045836,$20 to 50 Million,NC,10+ years,0
216356,5715 PACIFIC HWY E,52220000,FIFE,10 to 19,Nicole Hedges - REALTOR - Russ Lyon Sotheby's ...,7134731000,"Less Than $500,000",WA,6-10 years,0
236682,310 SOMERVILLE AVE,23610000,MORGAN,10 to 19,Michael Murphy MFT,3018556200,None,MN,3-5 years,0
263472,1275 SHILOH RD NW STE 2510,81149000,KENNESAW,5 to 9,BEYOND ACCOUNTING-BOOKKEEPING,8508629500,$1 to 2.5 Million,GA,10+ years,0
376455,5814 WILLIAMS DR,54121000,CORPUS CHRISTI,1 to 4,Wahlrich Group LLC,None,"Less Than $500,000",TX,10+ years,0
382725,7570 MYRTLE AVE,72200000,EUREKA,20 to 49,Heartfelt Images,None,"Less Than $500,000",CA,10+ years,0
396415,102 E MAIN ST STE 4,81311020,GEORGETOWN,5 to 9,STOTTS LAW PLLC,6128012969,$10 to 20 Million,KY,10+ years,0
431253,48 TROUT RUN MOUNTAIN RD,61111000,TROUT RUN,20 to 49,M & K THERAPY ASSOCIATE,None,None,PA,10+ years,0


In [22]:
#when zip is None it is clearly invalid
df_zip_none

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
19529,19555 GLENVIEW CT,54111000,OREGON CITY,50 to 99,Wilklow Orchards,None,"Less Than $500,000",OR,10+ years,None
35388,8101 S 15TH ST,52411000,LINCOLN,50 to 99,Protech Networks Inc,8606880888,$5 to 10 Million,NE,None,None
98992,2975 TREAT BLVD STE B5,54180000,CONCORD,1 to 4,HELP YOU WELL,None,"Less Than $500,000",CA,10+ years,None
197658,5601 FISHERS LN,62400000,ROCKVILLE,1 to 4,Spenard Builders Supply,8564641883,$1 to 2.5 Million,MD,10+ years,None
380630,656 S DOUGLAS ST,81130000,RIPON,1 to 4,Christian Board Of Publication,9496793991,"$500,000 to $1 Million",WI,10+ years,None
739431,2816 HOLLY DR,92100000,TRACY,100 to 249,Pomerance Sheryl DDS,9073762475,None,CA,10+ years,None
801765,110 COURTHOUSE SQ,92119000,BAY MINETTE,1 to 4,Bae Acupuncture,5035549000,"Less Than $500,000",AL,3-5 years,None
814638,7807 LONG POINT RD STE 216,52200000,HOUSTON,1 to 4,Covenant House Georgia,8182495707,"Less Than $500,000",TX,10+ years,None
834673,4700 LINCOLN AVE,56000000,CYPRESS,10 to 19,Day Realty Company,9528546565,$10 to 20 Million,CA,10+ years,None
836173,4228 MAIN ST,54161200,PHILADELPHIA,20 to 49,Goshen Town Public Works Department,7183401409,"$500,000 to $1 Million",PA,10+ years,None


In [47]:
#when zip is str, I need to check if it has 5 numeric characters'
#I write a function that first checks if 'zip' is str and second if it has 5 numeric characters.

def is_zip_valid(x):
    '''it checks if a zip value is a string with 5 numeric character'''
    
    if type(x) not in [type('str')]:
        return False
    
    re_zip_5digit = re.compile('^\d{5}$')
    if re_zip_5digit.match(x):
        return True
    else:
        return False

the number of records with valid zip is 953374


In [61]:
print ('the number of records with valid zip is ' + str(len(df[df['zip'].apply(is_zip_valid)])))

the number of records with valid zip is 953374


###### Count of true-valued for city

In [33]:
#check type of city values in the data frame
set([type(c) for c in list(df['city'].unique())])

{NoneType, int, str}

In [59]:
def is_city_str_valid(x):
    '''this function returns if an object a valid city name '''
    if type(x) not in [type('city')]:
        return False
    elif x in ['', 'none', 'None', ' ']:
        return False
    #I have exluded all the objects that have length less than 2. They are probably not 
    #a valid city name 
    elif len(x) < 2:
        return False
    else:
        return True

In [60]:
print ('the number of valid citis is ' + str(len(df[df['city'].apply(is_city_str_valid)])))

the number of valid citis is 999910


###### Count of true-valued category_code

Les first check the type of objects in category_code column.

In [62]:
set([type(cc) for cc in list(df['category_code'].unique())])

{int, NoneType, str}

In [63]:
def is_category_str(x):
    '''selects the rows where category code is string'''
    if type(x) not in [type('category')]:
        return False
    else:
        return True

df_category_str = df[df['category_code'].apply(is_category_str)]

In [64]:
def category_len(x):
    return len(x)

df_category_str['category_code'].apply(category_len).unique()

array([8, 1, 0, 4])

In [74]:
#category codes that are string with len 0. These are not valid country code
df_category_str_0 = df_category_str[df_category_str['category_code'].apply(lambda x: len(x) == 0)]                        
df_category_str_0['category_code'].value_counts()

    11
Name: category_code, dtype: int64

In [75]:
#category codes that are string with len 1. These are not valid country code
df_category_str_1 = df_category_str[df_category_str['category_code'].apply(lambda x: len(x) == 1)]
df_category_str_1['category_code'].value_counts()

     15
0    14
Name: category_code, dtype: int64

In [76]:
#category codes that are string with len 4. These are not valid country code
df_category_str_4 = df_category_str[df_category_str['category_code'].apply(lambda x: len(x) == 4)]
df_category_str_4['category_code'].value_counts()

null    14
none     9
Name: category_code, dtype: int64

In [77]:
#category codes that are string with len 8. These are potentially valid country code
df_category_str_8 = df_category_str[df_category_str['category_code'].apply(lambda x: len(x) == 8)]
df_category_str_8['category_code'].value_counts()

61111000    39461
54111000    37891
53120000    32577
54000000    31745
42000000    30171
62110000    29205
54100000    28693
23000000    27538
52420000    26887
62400000    22004
23820000    19235
62000000    19143
72200000    17026
81300000    16514
54121000    16343
62130000    15243
51000000    14867
44000000    14114
23611600    13293
56000000    12160
45000000    11590
52390000    10816
54161800     9504
52200000     9473
71300000     9447
23610000     9323
44400000     8544
81211000     8273
81111000     8142
52412000     7765
            ...  
31523200        1
44211025        1
49312001        1
23821014        1
33261200        1
62331203        1
44831006        1
48711003        1
33911308        1
61169918        1
21239200        1
33522401        1
54193002        1
32742001        1
81411002        1
48422009        1
56111008        1
62111174        1
31132000        1
44814006        1
72131000        1
45399832        1
31621100        1
33599905        1
81149009  

In [82]:
def is_valid_category_code(x):
    '''it checks if an object is a valid country code. In order to be valid it has 
    to be string with 8 numeric characters'''
    if type(x) not in [type('category')]:
        return False
    
    re_cat_8digit = re.compile('^\d{8}$')
    if re_cat_8digit.match(x):
        return True
    else:
        return False

In [84]:
print ('the number of records with valid category code is ' + str(len(df[df['category_code'].apply(is_valid_category_code)])))

the number of records with valid category code is 999910


###### Count of true-valued headcount

In [89]:
df['headcount'].unique()

array(['50 to 99', '1 to 4', '5 to 9', '10 to 19', '20 to 49', None,
       '100 to 249', '250 to 499', '500 to 999', 'Over 1,000', '0', 'none',
       0, 'null', ' ', ''], dtype=object)

In [148]:
def is_valid_headcount(x):
    '''select records of a dataframe that have valid headcount'''
    #I have made the assumption that headcount = 0 and '0' are not meaningfull
    valid_headcounts = ['50 to 99', '1 to 4', '5 to 9', '10 to 19', '20 to 49'
       '100 to 249', '250 to 499', '500 to 999', 'Over 1,000']
    return x[x['headcount'].isin(valid_headcounts)]

In [149]:
print ('the number of valid headcounts are ' + str(len(is_valid_headcount(df))))

the number of valid headcounts are 804534


###### Count of true-valued revenue

In [92]:
df['revenue'].unique()

array(['$20 to 50 Million', 'Less Than $500,000', '$500,000 to $1 Million',
       '$2.5 to 5 Million', '$1 to 2.5 Million', None, '$5 to 10 Million',
       '$10 to 20 Million', '$50 to 100 Million', '$100 to 500 Million',
       'Over $1 Billion', 'Over $500 Million', 'none', 'null', '0', 0, '',
       ' '], dtype=object)

In [155]:
def is_valid_revenue(x):
    '''select records of a dataframe that has valid revenus. I have made the assumption that
    companies with revenue = 0 are still valid'''
    
    valid_revenues = ['$20 to 50 Million', 'Less Than $500,000', '$500,000 to $1 Million',
       '$2.5 to 5 Million', '$1 to 2.5 Million', '$5 to 10 Million',
       '$10 to 20 Million', '$50 to 100 Million', '$100 to 500 Million',
       'Over $1 Billion', 'Over $500 Million', '0', 0]
    return x[x['revenue'].isin(valid_revenues)]

In [157]:
print ('the number of valid revenues are ' + str(len(is_valid_revenue(df))))

the number of valid revenues are 943031


###### Count of true-values time in business

In [95]:
df['time_in_business'].unique()

array(['10+ years', '6-10 years', None, '1-2 years', '3-5 years', 'null',
       '0', 'Less than a year', ' ', '', 'none', 0], dtype=object)

In [98]:
def time_in_business_valid_cnt(x):
    '''select records of a data frame that have valid time in buniness.
    I have made the assumption that 0 and is not a valid values for time in business'''
    
    valid_time_in_business = ['10+ years', '6-10 years', '1-2 years', '3-5 years', 'Less than a year']
    
    return x[x['time_in_business'].isin(valid_time_in_business)]


In [99]:
print ('the number of companies with valid time in business are ' + str(len(time_in_business_valid_cnt(df))))

the number of companies with valid time in business are 916048


###### Count of true-valued name 

In [100]:
set([type(p) for p in list(df['name'].unique())])

{NoneType, int, str}

In [101]:
def is_name_valid(x):
    '''identifies whether or not an object is a valid name for a company'''
    if type(x) not in [type('name')]:
        return False
    # I have excluded all the companies with name of length one.
    elif len(x) <= 1:
        return False
    else:
        return True


df_name_valid = df[df['name'].apply(is_name_valid)]

In [102]:
print ('number of companies with valid names are ' + str(len(df_name_valid)))

number of companies with valid names are 999931


###### Count of true-valued addresses in business

In [103]:
set([type(a) for a in list(df['address'].unique())])

{int, NoneType, str}

In [105]:
def is_address_valid(x):
    
    '''identifies if an object is valid for the address field.
    to be a valid address it has to be
    1- str
    2- starts with number
    3- is followed by at least by one alphabetic character
    . Obvisuly there might be other characteristics for a valid address, but at this point 
    I thought these are some reasonable priliminary filters'''
    
    if type(x) not in [type('str')]:
        return False
    
    re_house_number = re.compile('^[0-9]+')
    re_has_letter = re.compile('^.*[a-zA-Z].*')
    if re_house_number.match(x) and re_has_letter.match(x):
        return True
    else:
        return False

In [106]:
df_address_valid = df[df['address'].apply(is_address_valid)]               
print ('number of valid addresses are ' + str(len(df_address_valid)))

number of valid addresses are 999898


###### Count of True-values phone numbers

In [107]:
set([type(p) for p in list(df['phone'].unique())])

{int, NoneType, str}

In [111]:
#these are all phone numbers that are integer or None and are invalid
df[df['phone'].apply(lambda x: type (x) in [type(1)])]

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
93184,3412 FAIRFIELD AVE,62110000,FORT WAYNE,5 to 9,Eagle Dahlia Farm,0,$2.5 to 5 Million,IN,10+ years,46807
120179,85 MARION ST,52390000,NYACK,5 to 9,Autohaus Munich,0,$1 to 2.5 Million,NY,10+ years,10960
178010,28800 GODDARD RD,72200000,ROMULUS,1 to 4,Century Aluminum Company,0,$2.5 to 5 Million,MI,10+ years,48174
212302,21021 N 21ST ST,54121000,BARRINGTON,1 to 4,Anziano Technologies Inc,0,$100 to 500 Million,IL,10+ years,60010
317405,305 E COLORADO BLVD,44400000,DALLAS,None,Frisbie Sign Co,0,"Less Than $500,000",TX,10+ years,75203
333129,3102 SAN AUGUSTINE AVE,44400000,PASADENA,20 to 49,Korean Community Services Of Metropolitan,0,$1 to 2.5 Million,TX,10+ years,77503
386690,3700 LAS VEGAS BLVD S,81211000,LAS VEGAS,1 to 4,JOLAN TOREN,0,$2.5 to 5 Million,NV,10+ years,89109
472236,900 REDWOOD HWY FRONTAGE RD,48850000,MILL VALLEY,100 to 249,ADAMS CONSULTING GROUP LLC,0,"$500,000 to $1 Million",CA,10+ years,94941
708553,2635 RIVER RD,81331000,WILLOUGHBY HILLS,10 to 19,Bassett Home Furnishings,0,"Less Than $500,000",OH,10+ years,44094
723517,301 EISNER RD,53000000,CHAMPAIGN,10 to 19,KENOSHA UNIFIED SCHOOL DISTRICT 1,0,$5 to 10 Million,IL,10+ years,61822


In [113]:
#these are all phone numbers that are None and are invalid
df[df['phone'].apply(lambda x: type (x) in [type(None)])].head()

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
1,2566 SHALLOWFORD RD NE STE 104 # 302,31490000,ATLANTA,1 to 4,Real Hope Real Estate Inc,None,"Less Than $500,000",GA,10+ years,30345
5,1521 AZALEA RD,23610000,MOBILE,5 to 9,Venzon Engineering,None,$2.5 to 5 Million,AL,6-10 years,36693
7,178 BOOTHBAY RD,23611600,EDGECOMB,1 to 4,Discount Hauling LLC,None,"Less Than $500,000",ME,10+ years,04556
8,7 HAZZARD ST,56000000,WEST PALM BEACH,1 to 4,Knight Equestrian Books,None,"Less Than $500,000",FL,10+ years,33406
13,9200 KEYSTONE XING STE 150,54000000,INDIANAPOLIS,20 to 49,St Mary Magdalen Parish,None,"Less Than $500,000",IN,1-2 years,46240


Based on above observations, just string objects are potentially valid phone numbers. Lets explore string objects and their length further.

In [115]:
def is_phone_str(x):
    if type(x) not in [type('phone')]:
        return False
    else: 
        return True
df_phone_str = df[df['phone'].apply(is_phone_str)]               

df_phone_str['phone_len'] = df_phone_str['phone'].apply(len)
#follwoing shows the frequency of phone number length for the records with string phone number.
df_phone_str['phone_len'].value_counts()

10    581380
14      9412
4         33
1         28
0         12
15         2
13         2
16         1
12         1
Name: phone_len, dtype: int64

Ater investigation into the possible phone number lengths, I found following acceptable patterns for phone number.
<br>
1- with 10 digits : 8187689801
<br>
2- with 12 digits : 720-746-2480
<br>
3- with 13 digits : (713)217-2772
<br>
4- with 14 digits : (919) 716-6625
<br>
5- with 15 digits : (303) 223 -5129
<br>
6- with 16 digits : (617) 912 - 4833
<br>
I wrote a regex to cover all these combinations.

In [117]:
def is_phone_valid(x):
    '''finds acceptable phone numbers'''
    pattern = re.compile('^\(*[1-9]\d{2}[ \-\)]*\d{3}[ \-]*\d{4}')
    if pattern.match(x):
        return True
    else:
        return False

df_phone_valid = df_phone_str[df_phone_str['phone'].apply(is_phone_valid)]

In [119]:
print ('count of records with valid phone numbers are '+ str(len(df_phone_valid)))

count of records with valid phone numbers are 590798


## Part 3: Cardinality of each field

###### Cardinality for state

In [178]:
valid_states = valid_state_cnt(df)['state'].unique()
valid_states

array(['CA', 'GA', 'MO', 'OH', 'AL', 'MI', 'ME', 'FL', 'NM', 'HI', 'IN',
       'NY', 'IA', 'KS', 'MD', 'NJ', 'VA', 'DC', 'OR', 'NC', 'TX', 'PA',
       'ID', 'WA', 'UT', 'TN', 'MA', 'CT', 'CO', 'AZ', 'MS', 'OK', 'KY',
       'LA', 'MN', 'ND', 'WV', 'WI', 'SC', 'NH', 'IL', 'MT', 'RI', 'AR',
       'NV', 'DE', 'WY', 'SD', 'NE', 'VT', 'AK', 'VI', 'PR'], dtype=object)

In [183]:
print('field state has {0} unique valid values'.format(len(valid_states)))

field state has 53 unique valid values


###### Cardinality for zip

In [184]:
valid_zips = df[df['zip'].apply(is_zip_valid)]['zip'].unique()
valid_zips

array(['92131', '30345', '64850', ..., '32445', '80864', '45761'], dtype=object)

In [185]:
print ('field zip has {0} unique valid values'.format(len(valid_zips)))

field zip has 24410 unique valid values


###### Cardinality for city

In [186]:
valid_cities = df[df['city'].apply(is_city_str_valid)]['city'].unique()
valid_cities

array(['SAN DIEGO', 'ATLANTA', 'NEOSHO', ..., 'BOWDLE', 'WODEN',
       'MILLFIELD'], dtype=object)

In [187]:
print ('field city has {0} unique valid values'.format(len(valid_cities)))

field city has 13715 unique valid values


###### Cardinality for category code

In [189]:
valid_categories = df[df['category_code'].apply(is_valid_category_code)]['category_code'].unique()
valid_categories

array(['44420000', '31490000', '53120000', ..., '11531001', '32742001',
       '62111163'], dtype=object)

In [190]:
print ('field category code has {0} unique valid values'.format(len(valid_categories)))

field category code has 1178 unique valid values


###### Cardinality of headcount

In [191]:
valid_headcounts = is_valid_headcount(df)['headcount'].unique()
valid_headcounts

array(['50 to 99', '1 to 4', '5 to 9', '10 to 19', '250 to 499',
       '500 to 999', 'Over 1,000'], dtype=object)

In [192]:
print ('field headcount has {0} unique headcount values'.format(len(valid_headcounts)))

field headcount has 7 unique headcount values


###### Cordinality of revenue

In [193]:
valid_revenues = is_valid_revenue(df)['revenue'].unique()
valid_revenues

array(['$20 to 50 Million', 'Less Than $500,000', '$500,000 to $1 Million',
       '$2.5 to 5 Million', '$1 to 2.5 Million', '$5 to 10 Million',
       '$10 to 20 Million', '$50 to 100 Million', '$100 to 500 Million',
       'Over $1 Billion', 'Over $500 Million', '0', 0], dtype=object)

Since revenue 0 is basically the same as revenue '0', I would count them as just one.

In [194]:
print ('field revenue has {0} unique categories for revenue'.format(len(valid_revenues)-1))

field revenue has 12 unique categories for revenue


###### Cordinality of time in business

In [195]:
valid_time_in_business = time_in_business_valid_cnt(df)['time_in_business'].unique()
valid_time_in_business

array(['10+ years', '6-10 years', '1-2 years', '3-5 years',
       'Less than a year'], dtype=object)

In [196]:
print ('field time in business has {0} unique valid range'.format(len(valid_time_in_business)))

field time in business has 5 unique valid range


###### Cordinality of the name

In [197]:
valid_names = df_name_valid['name'].unique()
valid_names

array(['AMD CUSTOM', 'Real Hope Real Estate Inc',
       'Jimmy Sexton Photography', ..., 'Eachwin Lp', 'INC Research',
       'The Magrino Agency'], dtype=object)

In [198]:
print ('field name has {0} unique valid values'.format(len(valid_names)))

field name has 890717 unique valid values


###### Cordinality of the address

In [199]:
valid_addresses = df_address_valid['address'].unique()
valid_addresses

array(['10085 SCRIPPS RANCH CT STE A',
       '2566 SHALLOWFORD RD NE STE 104 # 302', '212 E MAIN ST', ...,
       '8020 OLD ALEXANDRIA FERRY RD', '3025 BIG FLAT RD',
       '700 N STONE AVE'], dtype=object)

In [200]:
print ('field address has {0} unique valid values'.format(len(valid_addresses)))

field address has 892114 unique valid values


###### Cordinality of the phone

In [201]:
valid_phones = df_phone_valid['phone'].unique()
valid_phones

array(['3123628000', '4046331779', '4174513798', ..., '9722403710',
       '5207456012', '3084523531'], dtype=object)

In [202]:
print ('field phone has {0} unique valid values'.format(len(valid_phones)))

field phone has 575148 unique valid values


## Part 4: Something cool or interesting in the data

In [230]:
def valid_dataset(x):
    '''In order to find interesting patterns in data filtered the records where all the fields
    valid. This way we are left with just about 40% of all the records but I think it is enough
    to find some interesting facts'''
    
    filter1 = x['state'].isin(list(valid_states))
    filter2 = x['zip'].isin(list(valid_zips))
    filter3 = x['category_code'].isin(list(valid_categories))   
    filter4 = x['city'].isin(list(valid_cities))
    filter5 = x['headcount'].isin(list(valid_headcounts))
    filter6 = x['revenue'].isin(list(valid_revenues))
    filter7 = x['phone'].isin(list(valid_phones))
    filter8 = x['address'].isin(list(valid_addresses))
    filter9 = x['name'].isin(list(valid_names))
    filter10 = x['time_in_business'].isin(list(valid_time_in_business))
    final_filter = np.logical_and.reduce((filter1, filter2, filter3, filter4, filter5, filter6, filter7, filter8, filter9, filter10))
    return x[final_filter]

df_filtered = valid_dataset(df)

In [239]:
df_filtered['time_in_business'].value_counts()/len(df_filtered)

10+ years     0.828656
6-10 years    0.115943
3-5 years     0.041534
1-2 years     0.013868
Name: time_in_business, dtype: float64

1 - Around 83% of the data is related to companies that have been in business for 10+ year.

In [240]:
df_filtered['revenue'].value_counts()/len(df_filtered)

Less Than $500,000        0.348902
$1 to 2.5 Million         0.179710
$500,000 to $1 Million    0.162668
$2.5 to 5 Million         0.105932
$5 to 10 Million          0.089401
$10 to 20 Million         0.051055
$20 to 50 Million         0.034363
$50 to 100 Million        0.013847
$100 to 500 Million       0.010629
Over $1 Billion           0.001807
Over $500 Million         0.001656
0                         0.000018
0                         0.000010
Name: revenue, dtype: float64

2- Around 53% of the companies have less than 2.5 million revenue.

In [246]:
set([len(x) for x in list(df_filtered['category_code'])])

{8}

3- All the valid fileds in category code are 8 digit values. Apparently in NAICS system, all the US industries have 6-digit category code. Any code with more than 6 digits are probably specific to the agency that has created them. In our case we probably need to find some mapping to decide how these codes can be interpereted into true NAICs codes. 

In [247]:
df_filtered['headcount'].value_counts()/len(df_filtered)

1 to 4        0.445299
5 to 9        0.264252
10 to 19      0.187203
50 to 99      0.075562
250 to 499    0.014210
Over 1,000    0.006894
500 to 999    0.006580
Name: headcount, dtype: float64

4 - More than 60% of all the companies in this data set have less than 10 employees. This suggests that the majority of these companies are either start up or small busniesses.

In [248]:
len(df_filtered['name'].unique()) == len(df_filtered)

False

As you can see we have duplicate values in the field name. The question would be how we can create a unique identifier for each company. Would combination of name and state creates a unique identifier for each company? Lets examine.

In [257]:
state_name_comb = df_filtered.groupby(['name', 'state']).size().to_frame().reset_index()
state_name_comb.columns = ['name', 'state', 'count']
state_name_comb[state_name_comb['count'] > 1]

,name,state,count
53,1-800-GOT-JUNK,CA,2
57,1-800-GOT-JUNK?,FL,2
60,1-800-GOT-JUNK?,MI,2
63,1-800-GOT-JUNK?,OH,2
64,1-800-GOT-JUNK?,TX,2
660,3M COMPANY,CA,2
954,7-Eleven,CA,2
1081,@PROPERTIES,FL,2
1518,A B C UNIFIED SCHOOL DISTRICT,IA,2
1754,A Cut Above,CA,2


5- As you can see, even after combining name and state as an identifier for the company, we are not able to define unique identifiers. Based on above analysis, for some of the names we have count more than one in some of the states. 'YMCA', 'AAA', 'Zurich American Insurance Company' are some of the examples. These are mostly national companies in the US and operate across the state lines. I suggest some sort of aggregation in data manipulation and processing phase for such companies.